In [13]:
# Imports
load('prismatic_envelope.sage')
load('precision.py')

### User-defined input

In [19]:
# The prime p
p=2

# The motivic weight i
i=3

# The power of the uniformizer n
n=9

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [20]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='series',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 132
Fprec is 27


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [21]:
# The Eisenstein polynomial E
E=A(z+2)

### The main calculation

In [22]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

# The syntomic matrices from new
syn0,syn1,nablaN,nablaP=syntomic_matrices(p,i,n,E,total_precision,Fprec)

Defining f0, f1


<ipython-input-22-36442ace7685>:9: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See http://trac.sagemath.org/14825 for details.
  syn0,syn1,nablaN,nablaP=syntomic_matrices(p,i,n,E,total_precision,Fprec)


Defining d
Defining g0, g1, g2, g3, g4
((2^10 + 2^12 + 2^15 + 2^18 + 2^22 + 2^23 + 2^26 + 2^27 + 2^29 + 2^32 + 2^34 + 2^35 + 2^36 + 2^37 + 2^38 + 2^39 + 2^40 + 2^41 + 2^43 + 2^46 + 2^47 + 2^48 + 2^49 + 2^50 + 2^51 + 2^52 + 2^53 + 2^55 + 2^56 + 2^59 + 2^60 + 2^63 + 2^65 + 2^67 + 2^69 + 2^71 + 2^73 + 2^74 + 2^75 + 2^77 + 2^79 + 2^84 + 2^87 + 2^88 + 2^90 + 2^94 + 2^95 + 2^96 + 2^98 + 2^100 + 2^101 + 2^102 + 2^108 + 2^109 + 2^114 + 2^116 + 2^117 + 2^118 + 2^119 + 2^121)*z + (2^14 + 2^16 + 2^18 + 2^19 + 2^20 + 2^24 + 2^25 + 2^28 + 2^34 + 2^37 + 2^40 + 2^41 + 2^45 + 2^47 + 2^49 + 2^51 + 2^52 + 2^54 + 2^55 + 2^57 + 2^62 + 2^63 + 2^66 + 2^67 + 2^71 + 2^72 + 2^75 + 2^76 + 2^78 + 2^80 + 2^85 + 2^86 + 2^89 + 2^90 + 2^91 + 2^93 + 2^94 + 2^96 + 2^97 + 2^99 + 2^101 + 2^102 + 2^103 + 2^104 + 2^105 + 2^106 + 2^109 + 2^110 + 2^111 + 2^113 + 2^114 + 2^115 + 2^118 + 2^120)*z^2 + (2^10 + 2^13 + 2^14 + 2^18 + 2^19 + 2^22 + 2^24 + 2^25 + 2^27 + 2^33 + 2^34 + 2^36 + 2^38 + 2^39 + 2^40 + 2^43 + 2^47 + 2^51 + 

nablaP_OK is 

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Assembling the syntomic complex and computing its cohomology

In [23]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(syn0,syn1,nablaN,nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_4(R;Z_p) are [2^3]
Elementary divisors of K_5(R;Z_p) are [2, 2^2, 2^24]
Target precision is 46 and final precision is 64


In [7]:
syn0

[  2^2     0     0]
[1 + 2     2     0]
[    1     1     2]

In [8]:
nablaP

[                            1                             0                             0]
[                            0     2 + 2^2 + 2^4 + 2^6 + 2^8                             0]
[                2 + 2^2 + 2^3         2^2 + 2^3 + 2^4 + 2^8 1 + 2 + 2^2 + 2^3 + 2^7 + 2^8]

In [9]:
syn1

[            2             0             0]
[2 + 2^2 + 2^4             2             0]
[            2             1             1]

In [10]:
nablaN

[                  2                   0                   0]
[  1 + 2 + 2^3 + 2^5 2 + 2^2 + 2^4 + 2^6                   0]
[          2^2 + 2^3 1 + 2^4 + 2^5 + 2^6 2 + 2^2 + 2^3 + 2^4]

In [11]:
M=Matrix(ZZ,[[1,0],[0,1]])